## 最適ポートフォリオ


In [2]:
import pandas as pd
import numpy as np
import os
import glob
import datetime
import jpholiday

In [16]:
# ディレクトリ内のファイル一覧を取得
path = "/Volumes/GoogleDrive/マイドライブ/Univ/Finance/ETF_portfolio/portfolio_2021_3_3(1)/"
files = glob.glob( path + "*" )
file_list = []
for file in files:
    basename = os.path.basename(file)
    file_list.append(basename)
print("all csv file : ",file_list)

all csv file :  ['1460.csv', '1357.csv', '1489.csv', '1490.csv', '1312.csv']


In [17]:
# スクレイピングした株価データから日付，終値だけのデータフレームを生成
data_csv = []
for f in file_list:
    print(f,end="")
    df = pd.read_csv( path + f ,index_col=0)
    df_i = df.set_index('日付')         #日付をインデックスとして指定
    df_i = df_i["終値"]
    data_csv.append(df_i)
    print(" : length =" , len(df))
print(data_csv[2])

1460.csv : length = 572
1357.csv : length = 1488
1489.csv : length = 974
1490.csv : length = 420
1312.csv : length = 1267
日付
2017-02-13    36000.0
2017-02-14    35850.0
2017-02-15    36400.0
2017-02-16    36200.0
2017-02-17    36250.0
               ...   
2021-02-03    31850.0
2021-02-12    33100.0
2021-02-24    33550.0
2021-02-26    33100.0
2021-03-03    33950.0
Name: 終値, Length: 974, dtype: float64


In [18]:
# スクレイピングした株価データよみこみ ================================================================
data_csv = []
for f in file_list:
    df = pd.read_csv( path + f )
    data_csv.append(df)

# 複数のcsvの中で一番データ数の多いデータから市場の開場日一覧を取得 ==================================================
length = []
for df in data_csv:
    leng = len(df)
    length.append(leng) 
date_len = length.index(max(length))
df_date = data_csv[date_len]
date = df_date["日付"]


# スクレイピングした株価データから日付，終値だけのデータフレームを生成 ================================
data_csv = []
for f in file_list:
    #print(f,end="")
    df = pd.read_csv( path + f ,index_col=0)
    df_i = df.set_index('日付')         #日付をインデックスとして指定
    df_i = df_i["終値"]
    data_csv.append(df_i)


# date[0] == "2015-10-20" & data_csv は日付をインデックスとして指定しているので以下の2つの参照文は同じ結果を返す．
#print(data_csv[2]["2015-10-20"])
#print(data_csv[2][date[0]])

# 株価欠損値をnanで表現 ================================================================================
# (1) data_csv中からdfを一つとってくる 
#     data_stock = pd.dataframeを作成 (欠損値処理後のデータ組のを格納するdataframe)
#     d = []作成 (dfに欠損値処理したあとのデータを格納するリスト)
data_stock = pd.DataFrame()
data_stock["date"] = date       #一列目に日付インデックスを入れる
for i in range(len(data_csv)):
    df = data_csv[i]
    d =[]
# (2) dateから日付インデックスyyyymmddでfor文
#     try文でdf[date]を取得
#        要素があったらdata_priceに格納
#           要素が複数あったら例外処理(データ型が配列になっているか数値かで判定)
#     except文：なかったら'nan'をdata_priceに格納
    for yyyymmdd in date:
        try:
            df[yyyymmdd]
            # csv内で同じ日付の要素が複数あったときの処理
            # df[yyyymmdd]の型がpd.seriesかfloatかで判別
            if type(df[yyyymmdd]) == pd.core.series.Series:
                # データ型がpd.seriesならseriesの1つ目の要素を取得
                data_yyyymmdd = df[yyyymmdd][0]
            else:
                # データ型がfloatならそのまま取得
                data_yyyymmdd = df[yyyymmdd]
        except KeyError:
            # 存在しない日付データはnanとして取得
            data_yyyymmdd = np.nan
        d.append(data_yyyymmdd)
    data_stock[str(file_list[i])] = d
# 日付をインデックスに指定
data_stock = data_stock.set_index('date')
# csvに保存
data_stock.to_csv(path + "data_stock.csv")
print(data_stock)

            1460.csv  1357.csv  1489.csv  1490.csv  1312.csv
date                                                        
2015-01-05       NaN    3835.0       NaN       NaN   15610.0
2015-01-06       NaN    4060.0       NaN       NaN   15400.0
2015-01-07       NaN    4050.0       NaN       NaN   15310.0
2015-01-08       NaN    3915.0       NaN       NaN   15210.0
2015-01-09       NaN    3905.0       NaN       NaN       NaN
...              ...       ...       ...       ...       ...
2021-02-03       NaN     449.0   31850.0       NaN   20980.0
2021-02-12       NaN     422.0   33100.0    7730.0   21440.0
2021-02-24       NaN     415.0   33550.0       NaN   21140.0
2021-02-26   23570.0     432.0   33100.0       NaN   20800.0
2021-03-03   23010.0     416.0   33950.0       NaN   21200.0

[1488 rows x 5 columns]


In [19]:
# 収益率の計算をする関数
def cal_return(data):
    ret = [np.nan]
    for i in range(1,len(data)):
        ret.append( (data[i] - data[i-1])/data[i-1] )
    return(ret)

# データ全てに対して収益率の計算
return_data = pd.DataFrame()
return_data["date"] = date
for i in range(len(file_list)):
    fname = file_list[i]
    data = data_stock[fname]
    ret = cal_return(data)
    return_data[fname] = ret
return_data = return_data.set_index('date')
return_data.to_csv(path + "data_return.csv")
print(return_data)

            1460.csv  1357.csv  1489.csv  1490.csv  1312.csv
date                                                        
2015-01-05       NaN       NaN       NaN       NaN       NaN
2015-01-06       NaN  0.058670       NaN       NaN -0.013453
2015-01-07       NaN -0.002463       NaN       NaN -0.005844
2015-01-08       NaN -0.033333       NaN       NaN -0.006532
2015-01-09       NaN -0.002554       NaN       NaN       NaN
...              ...       ...       ...       ...       ...
2021-02-03       NaN -0.021786  0.015949       NaN  0.008169
2021-02-12       NaN -0.060134  0.039246       NaN  0.021926
2021-02-24       NaN -0.016588  0.013595       NaN -0.013993
2021-02-26       NaN  0.040964 -0.013413       NaN -0.016083
2021-03-03 -0.023759 -0.037037  0.025680       NaN  0.019231

[1488 rows x 5 columns]


## 平均分散モデル
次の二次計画問題  
  
$ \min\   \omega ^T \Sigma \omega $  
$\mbox{s.t} \   
\omega^T \mu = \mu_p \  ,\  
\omega^T l = 1
$  
  
の解
$
\Rightarrow \omega_p = \dfrac{C \mu_p - A}{D}\Sigma^{-1} \mu + \dfrac{B-A\mu_p}{D}\Sigma^{-1}l
$  
を計算する．ただし，  
$
\begin{cases}
A = \mu^T \Sigma^{-1} l \\
B = \mu^T \Sigma^{-1} \mu \\
C = l^T \Sigma^{-1} l \\
D = l^T \Sigma^{-1} \mu = BC-A^2 \\
\mu_p:\mbox{目標とする期待収益率} \\
\omega_p:\mbox{最小分散ポートフォリオの投資比率}
\end{cases}
$

In [46]:
# 平均分散モデルで分散最小化 =======================================================
# 平均分散モデルの解を計算する関数
def optimal_portfolio(data, return_rate):
    # 分散共分散行列をndarrayに変換して逆行列を計算
    sigma = data.cov()
    sigma = sigma.values
    sigma_inv = np.linalg.inv(sigma)
    # 期待収益率ベクトルを計算
    mu = data.mean()
    mu = mu.values
    # 成分がすべて1で長さが期待収益率ベクトルと等しいベクトル l を生成
    l = np.ones(len(mu))
    # A,B,C,Dを計算
    A = np.dot(mu, np.dot(sigma_inv, l) )
    B = np.dot(mu, np.dot(sigma_inv, mu))
    C = np.dot(l, np.dot(sigma_inv, l))
    D = B*C-A*A

    # 最適投資比率 omega を計算
    omega = ((C*return_rate - A)/D)*np.dot(sigma_inv, mu) + ( (B - A*return_rate)/D )*np.dot(sigma_inv, l)

    return(omega)

# データから最適投資比率を計算 =======================================================
omega_p = optimal_portfolio(return_data, 0.002)
S = sum(omega_p)
#omega_p =omega_p /  S
print(omega_p)

[ 1.6116873   0.59780314  1.22950021 -3.41282026  0.97382961]


In [44]:
S = 0
for w in omega_p:
    S+=w
print(S)
print(omega_p[2]+1)

0.9999999999999716
38.300900520427675


## 練習

In [ ]:
# 例外処理の練習 =======================================================

# 例) 例外処理込みでdfの要素表示
for yyyymmdd in date:
    print(yyyymmdd)
    df = data_csv[0]
    try:
        print(df[yyyymmdd])
    except KeyError:
        print("nan")

# 例) data_csv[0]を欠損値込み形式にする
df = data_csv[0]
data_price_0 = []
for yyyymmdd in date:
    try:
        data_yyyymmdd = df[yyyymmdd]
        # csv内で同じ日付の要素が複数あったときの処理
        # df[yyyymmdd]の型がリストpd.seriesかfloatかで判別
        if type(df[yyyymmdd]) == pd.core.series.Series:
            # データ型がpd.seriesならseriesの1つ目の要素を取得
            data_yyyymmdd = df[yyyymmdd][0]
        else:
            # データ型がfloatならそのまま取得
            data_yyyymmdd = df[yyyymmdd]
    except KeyError:
        # 存在しない日付の株式データはnanとして取得
        data_yyyymmdd = np.nan
    data_price_0.append(data_yyyymmdd)

In [8]:
#for i in range(len(data_price_0)):
#    print(date[i])
#    print(data_price_0[i])
print(len(data_price_0))
print(data_price_0[1264])
print(type(data_price_0[1264]))
#for i in range(1250,1277):
#   print(i, ":", data_price_0[i])
print(df["2021-01-04"][0])
print(type(df["2021-01-04"])==pd.core.series.Series)

1290
日付
2021-01-04    1818.0
2021-01-04    1818.0
2021-01-04    1818.0
2021-01-04    1818.0
2021-01-04    1818.0
2021-01-04    1818.0
Name: 終値, dtype: float64
<class 'pandas.core.series.Series'>
1818.0
True


In [38]:
# dataframe作成の練習

# リストから作成
lst = ["Jay","Raj","Jack"]
df = pd.DataFrame(lst, columns = ['Name'])
#print(df)

lst1 = ["Jay","Raj","Jack"]
lst2 = [12,15,14]
df = pd.DataFrame(list(zip(lst1,lst2)), columns = ['Name','Age'])
#print(df)


#df = pd.DataFrame(list(zip(data_csv[0],data_csv[1])))
#df = pd.DataFrame( list( data_csv ) ) 
index = date.values
print(type(index))
#df = pd.DataFrame(data_csv[0], index=index)
print(df[1276:1280])

<class 'numpy.ndarray'>
Empty DataFrame
Columns: [Name, Age]
Index: []


In [60]:
# 欠損値処理の練習
df_t = pd.read_csv( "test.csv" ,index_col=0)
df_t = df_t["数字"]
print(df_t)
print(type(df_t[0]))
print(type(df_t[1]))
print(type(np.nan))
print(df_t[0] + df_t[1])
print(type(df_t[0] + df_t[1]))

日付
1900/1/1    2.0
1900/4/7    NaN
Name: 数字, dtype: float64
<class 'numpy.float64'>
<class 'numpy.float64'>
<class 'float'>
nan
<class 'numpy.float64'>


In [45]:
test = pd.DataFrame()
test["date"] = date
test["1577"] = data_price_0 
print(test)

            date    1577
0     2015-10-20  1670.0
1     2015-10-21  1700.0
2     2015-10-22  1705.0
3     2015-10-23  1735.0
4     2015-10-26  1750.0
...          ...     ...
1285  2021-01-29  1824.0
1286  2021-02-01  1833.0
1287  2021-02-02  1839.0
1288  2021-02-03  1862.0
1289  2021-02-12  1888.0

[1290 rows x 2 columns]


In [52]:
# 株価欠損値をnanで表現 ================================================================================
# (1) data_csv中からdfを一つとってくる 
#     data_stock = []作成 (欠損値処理を組み入れたデータを格納するリスト)
#     d = []作成 (dfに欠損値処理したあとのデータを格納するリスト)
data_stock = pd.DataFrame()
data_stock["date"] = date
for i in range(len(data_csv)):
    df = data_csv[i]
    d =[]
# (2) dateから日付インデックスyyyymmddでfor文
#     try文でdf[date]を取得
#        要素があったらdata_priceに格納
#           要素が複数あったら例外処理(データ型が配列になっているか数値かで判定)
#     except文：なかったら'nan'をdata_priceに格納
    for yyyymmdd in date:
        try:
            df[yyyymmdd]
            # csv内で同じ日付の要素が複数あったときの処理
            # df[yyyymmdd]の型がリストpd.seriesかfloatかで判別
            if type(df[yyyymmdd]) == pd.core.series.Series:
                # データ型がpd.seriesならseriesの1つ目の要素を取得
                data_yyyymmdd = df[yyyymmdd][0]
            else:
                # データ型がfloatならそのまま取得
                data_yyyymmdd = df[yyyymmdd]
        except KeyError:
            # 存在しない日付データはnanとして取得
            data_yyyymmdd = np.nan
        d.append(data_yyyymmdd)
    data_stock[str(file_list[i])] = d
print(data_stock)

            date  1477.csv  1478.csv  1475.csv
0     2015-10-20    1670.0    1820.0    1501.0
1     2015-10-21    1700.0    1865.0    1527.0
2     2015-10-22    1705.0    1860.0    1520.0
3     2015-10-23    1735.0    1903.0    1549.0
4     2015-10-26    1750.0    1909.0    1560.0
...          ...       ...       ...       ...
1285  2021-01-29    1824.0    1978.0    1848.0
1286  2021-02-01    1833.0    1989.0    1875.0
1287  2021-02-02    1839.0    2002.0    1891.0
1288  2021-02-03    1862.0    2048.0    1917.0
1289  2021-02-12    1888.0    2084.0    1966.0

[1290 rows x 4 columns]
